<a href="https://colab.research.google.com/github/viniciusrpb/cic0193_machinelearning/blob/main/cap9_2_first_cnns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capítulo 9

### 9.2. Criando Redes Neurais Convolucionais


In [1]:
from keras.models import Sequential
from keras.layers import Dense,MaxPooling2D,Activation,Flatten,Conv2D,BatchNormalization,Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from sklearn.metrics import classification_report
import numpy as np

Carregando o conjunto de dados de folhas que utilizaremos como exemplo:

### 9.2.2. Utilizando nossos próprios dados




In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp -r "/content/drive/My Drive/leafs" "leafs"

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range =20,
                                   fill_mode='nearest',
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255.)

train_set = train_datagen.flow_from_directory("leafs/train",
                                              subset='training',
                                              target_size=(128,128),
                                              class_mode="categorical",
                                              batch_size=32)

validation_set = train_datagen.flow_from_directory("leafs/train",
                                              subset='validation',
                                              target_size=(128,128),
                                              class_mode="categorical",
                                              batch_size=32)

test_set = test_datagen.flow_from_directory("leafs/test",
                                            target_size=(128,128),
                                            class_mode="categorical",
                                            batch_size=32)

Found 160 images belonging to 4 classes.
Found 40 images belonging to 4 classes.
Found 100 images belonging to 4 classes.


In [5]:
kernel_dimensions = (5,5)
tam_strides = (1,1)
number_filters = 50
number_of_neurons = 30

num_classes = 4

In [11]:
model = Sequential()

model.add(Conv2D(filters=number_filters,padding="same",kernel_size=kernel_dimensions,input_shape=(128,128,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(filters=25,padding="same",kernel_size=kernel_dimensions))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(filters=15,padding="same",kernel_size=kernel_dimensions))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(number_of_neurons))
model.add(Activation("relu"))
#model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation("softmax"))


In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 128, 128, 50)      3800      
_________________________________________________________________
activation_5 (Activation)    (None, 128, 128, 50)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 64, 50)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 25)        31275     
_________________________________________________________________
activation_6 (Activation)    (None, 64, 64, 25)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 25)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 15)       

In [13]:
sgd = SGD(learning_rate=0.1)

model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [14]:
model.fit(train_set,validation_data=validation_set,epochs=20)

Epoch 1/20
5/5 [==============================] - 16s 3s/step - loss: 1.4417 - accuracy: 0.2062 - val_loss: 1.3864 - val_accuracy: 0.3000
Epoch 2/20
5/5 [==============================] - 13s 3s/step - loss: 1.3890 - accuracy: 0.2375 - val_loss: 1.3867 - val_accuracy: 0.2250
Epoch 3/20
5/5 [==============================] - 13s 3s/step - loss: 1.3955 - accuracy: 0.2125 - val_loss: 1.3852 - val_accuracy: 0.2250
Epoch 4/20
5/5 [==============================] - 13s 3s/step - loss: 1.3873 - accuracy: 0.2438 - val_loss: 1.3849 - val_accuracy: 0.3500
Epoch 5/20
5/5 [==============================] - 13s 3s/step - loss: 1.3868 - accuracy: 0.2625 - val_loss: 1.3836 - val_accuracy: 0.3750
Epoch 6/20
5/5 [==============================] - 13s 3s/step - loss: 1.3858 - accuracy: 0.2125 - val_loss: 1.3840 - val_accuracy: 0.2750
Epoch 7/20
5/5 [==============================] - 13s 3s/step - loss: 1.3852 - accuracy: 0.2438 - val_loss: 1.3788 - val_accuracy: 0.3250
Epoch 8/20
5/5 [==================

In [15]:
y_prob = model.predict(test_set)
y_pred = np.argmax(y_prob,axis=1)

print(classification_report(test_set.classes,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        25
           1       0.24      0.88      0.38        25
           2       0.50      0.08      0.14        25
           3       0.33      0.08      0.13        25

    accuracy                           0.26       100
   macro avg       0.27      0.26      0.16       100
weighted avg       0.27      0.26      0.16       100



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Jargões

Stride: pulo da evolução no processo de convolução entre uma imagem e um fitlro. O ideal é utilizarmos stride 1 ou 2 (caso a imagem seja grande). No stride 1, cada pixel da imagem é considerado na convolução, enquanto que no stride 2, o filtro é passado sobre a imagem a cada 2 pixels.

Stride horizontal Stride vertical